In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
from src.data import constants
from src.data.ancestry import load_train_test
from src.metrics import metrics
from src.models import utils
from src.models.autoencoder import AutoEncoder, train_model

In [ ]:
MAX_NAME_LENGTH = 30

### Load data

In [ ]:
train, test = load_train_test(f'../data/raw/records25k_data_train.csv',
                              f'../data/raw/records25k_data_test.csv')
_, _, candidate_names_train = train
input_names_test, weighted_actual_names_test, candidate_names_test = test

actual_names_test = [[name for name, _, _ in name_weights] for name_weights in weighted_actual_names_test]

candidate_names_all = np.concatenate((candidate_names_train, candidate_names_test))

### Build token index mappings

In [ ]:
char_to_idx_map, idx_to_char_map = utils.build_token_idx_maps()

### Convert names to ids

In [ ]:
# Prepare data for training
# inputs and targets have the same data just in different representations 1-hot vs normal sequences
candidate_names_train_X, candidate_names_train_y = utils.convert_names_to_model_inputs(candidate_names_train,
                                                                                    char_to_idx_map,
                                                                                    MAX_NAME_LENGTH)

In [ ]:
print(candidate_names_train_X.shape, candidate_names_train_y.shape)

### Model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model = AutoEncoder(input_size=constants.VOCAB_SIZE + 1,
                    hidden_size=100,
                    num_layers=1,
                    seq_len=MAX_NAME_LENGTH,
                    device=device)

In [ ]:
train_model(model, candidate_names_train_X, candidate_names_train_y, 100, 512)

In [ ]:
torch.save(model, '../models/anc-encoder-bilstm-100-512.pth')

In [ ]:
model = torch.load('../models/anc-encoder-bilstm-100-512.pth')

### Understand AutoEncoder

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
dataset_train = torch.utils.data.TensorDataset(candidate_names_train_X, candidate_names_train_y)
data_loader = torch.utils.data.DataLoader(dataset_train, batch_size=512, shuffle=True)

In [ ]:
X, y = next(iter(data_loader))
print(X.shape, y.shape)

In [ ]:
model.zero_grad()
# Encode(input,hidden) -> (batch,seq,dirs*hidden), ((dirs*layers,batch,hidden),(dirs*layers,batch,hidden)) - x_encoded is the last hidden state
_, (x_encoded, _) = model.lstm_encoder(X.to(device))
print(x_encoded.shape)

In [ ]:
# Concatenate left-right hidden vectors
x_encoded = torch.cat([x_encoded[0], x_encoded[1]], dim=1)
print(x_encoded.shape)

In [ ]:
# Reshape data to have seq_len time steps
# TODO why do we copy x_encoded to every time step?
x_encoded = x_encoded.unsqueeze(1).repeat(1, MAX_NAME_LENGTH, 1)
print(x_encoded.shape)

In [ ]:
# Decode(hidden*dirs,hidden) -> (batch,seq,dirs*hidden), ((dirs*layers,batch,hidden),(dirs*layers,batch,hidden)) - x_decoded is the output
x_decoded, (_, _) = model.lstm_decoder(x_encoded)
print(x_decoded.shape)

In [ ]:
# linear layer(hidden,input) predicts characters
x_prime = model.linear(x_decoded)
print(x_prime.shape)

In [ ]:
# Reshape output to match CrossEntropyLoss input
x_prime = x_prime.transpose(1, -1)
print(x_prime.shape)

In [ ]:
# Compute loss (batch,classes,seq), (batch,seq)
loss = loss_fn(x_prime, y.to(device))
print(loss)

### Evaluation

In [ ]:
model.to("cpu")
model.device="cpu"

In [ ]:
# Get Embeddings for the names from the encoder
candidate_names_train_encoded = model(candidate_names_train_X, just_encoder=True).detach().numpy()

In [ ]:
print(candidate_names_train_X.shape)
print(candidate_names_train_encoded.shape)

In [ ]:
# Convert test name inputs to correct format
input_names_test_X, input_names_test_y = utils.convert_names_to_model_inputs(input_names_test,
                                                                          char_to_idx_map,
                                                                          MAX_NAME_LENGTH)
# Get Embeddings for the names from the encoder
input_names_test_encoded = model(input_names_test_X, just_encoder=True).detach().numpy()

In [ ]:
print(input_names_test_X.shape, input_names_test_y.shape)
print(input_names_test_encoded.shape)

In [ ]:
# Get embeddings for all candidate names (train + test)
candidate_names_test_X, _ = utils.convert_names_to_model_inputs(candidate_names_test,
                                                             char_to_idx_map,
                                                             MAX_NAME_LENGTH)
candidate_names_test_encoded = model(candidate_names_test_X, just_encoder=True).detach().numpy()

In [ ]:
print(candidate_names_test_X.shape)
print(candidate_names_test_encoded.shape)

In [ ]:
candidate_names_all_encoded = np.vstack((candidate_names_train_encoded, candidate_names_test_encoded))

In [ ]:
print(candidate_names_all_encoded.shape)

In [ ]:
k = 100
candidate_names_scores = utils.get_best_matches(input_names_test_encoded,
                                                candidate_names_all_encoded,
                                                candidate_names_all,
                                                num_candidates=k)
print(candidate_names_scores.shape)
print(candidate_names_scores[0, 0, 0])
print(candidate_names_scores[0, 0, 1])

In [ ]:
candidate_names = candidate_names_scores[:, :, 0]
print(candidate_names.shape)

### Evaluation

In [ ]:
metrics.precision_recall_curve_at_k(actual_names_test, candidate_names, k)

### mAP @ 1

In [ ]:
metrics.mean_avg_precision_k(actual_names_test, candidate_names, 1)

### mAP @ 3

In [ ]:
metrics.mean_avg_precision_k(actual_names_test, candidate_names, 3)

### Demo

In [ ]:
test_name_X = ['<schumacher>']
test_name_X, _ = utils.convert_names_to_model_inputs(test_name_X, char_to_idx_map, MAX_NAME_LENGTH)
test_name_embedding = model(test_name_X, just_encoder=True).detach().numpy()

print(utils.get_best_matches(test_name_embedding,
                             candidate_names_all_encoded,
                             candidate_names_all,
                             num_candidates=10))

## Evaluate using weighted relevant names and score thresholds

### Average precision at 0.97

In [ ]:
metrics.avg_precision_at_threshold(weighted_actual_names_test, candidate_names_scores, 0.97)

### Average recall at 0.97

In [ ]:
metrics.avg_weighted_recall_at_threshold(weighted_actual_names_test, candidate_names_scores, 0.97)

### PR Curve

In [ ]:
# minimum score threshold to test
min_threshold = 0.5
metrics.precision_weighted_recall_curve_at_threshold(weighted_actual_names_test, candidate_names_scores, min_threshold)

### AUC

In [ ]:
metrics.get_auc(weighted_actual_names_test, candidate_names_scores)